## Import necessary modules and data

In [5]:
import numpy as np
import pandas as pd
import sklearn
import keras
import seaborn
import matplotlib.pyplot as plt
import tensorflow
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
#!pip install -q keras_metrics
import keras_metrics
from sklearn.model_selection import cross_validate
#!pip install livelossplot
from livelossplot import PlotLossesKeras
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.metrics import precision_recall_fscore_support as score
import random


Using TensorFlow backend.


In [4]:
url = 'https://raw.githubusercontent.com/Dalthirr/NeuralNetworks/master/Churn_Modelling.csv'
#Pobieranie danych
df = pd.read_csv(url)
df = df.drop(["CustomerId", "Surname", "RowNumber"],
             axis=1)
df = pd.concat([df, pd.get_dummies(df[["Gender", "Geography"]], drop_first=True)],
               sort=False, axis=1).drop(df[["Gender", "Geography"]], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
CreditScore          10000 non-null int64
Age                  10000 non-null int64
Tenure               10000 non-null int64
Balance              10000 non-null float64
NumOfProducts        10000 non-null int64
HasCrCard            10000 non-null int64
IsActiveMember       10000 non-null int64
EstimatedSalary      10000 non-null float64
Exited               10000 non-null int64
Gender_Male          10000 non-null uint8
Geography_Germany    10000 non-null uint8
Geography_Spain      10000 non-null uint8
dtypes: float64(2), int64(7), uint8(3)
memory usage: 732.5 KB


## Tworzymy zbiór treningowy i testowy, standaryzujemy i stratyfikujemy odpowiednio do proporcji etykiet

In [6]:
X = df.drop(['Exited'], axis=1)
y = df['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

scaler = StandardScaler().fit(X_train)
X_train_std = scaler.transform(X_train)

X_test_std = scaler.transform(X_test)

X_std = scaler.transform(X)

/home/seba/.virtualenvs/rl/lib/python3.5/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/seba/.virtualenvs/rl/lib/python3.5/site-packages/ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  import sys
/home/seba/.virtualenvs/rl/lib/python3.5/site-packages/ipykernel_launcher.py:9: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  if __name__ == '__main__':
/home/seba/.virtualenvs/rl/lib/python3.5/site-packages/ipykernel_launcher.py:11: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  # This is added back by InteractiveShellApp.init_path()


## Paweł Bogdan

In [7]:
#Wczytuję dane - tym razem wyrzucam również informację odnośnie
#posiadania karty kredytowej - korelacja ze zmienną objaśnianą
#wynosiła 0.0003
df2 = pd.read_csv(url).drop(["CustomerId", "Surname", "RowNumber", "HasCrCard"],
                            axis=1)
df2 = pd.concat([df2, pd.get_dummies(df2[["Gender", "Geography"]], drop_first=True)],
               sort=False, axis=1).drop(df2[["Gender", "Geography"]], axis=1)

X2=df2.drop(['Exited'], axis=1)
y2=df2['Exited']

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42, stratify=y2)


scaler2 = StandardScaler().fit(X_train2)
X_train_std2 = scaler2.transform(X_train2)

X_test_std2 = scaler2.transform(X_test2)

X_std2 = scaler2.transform(X2)

#Poza standaryzacja i stratyfikacją zbioru treningowego i testowego
#ustalam również wagi dla danych klas
class_weight = {
                0: 3.,
                1: 1.
               }

es = EarlyStopping(monitor='val_precision', mode='max', verbose=1, patience=3)




/home/seba/.virtualenvs/rl/lib/python3.5/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/seba/.virtualenvs/rl/lib/python3.5/site-packages/ipykernel_launcher.py:16: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  app.launch_new_instance()
/home/seba/.virtualenvs/rl/lib/python3.5/site-packages/ipykernel_launcher.py:18: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
/home/seba/.virtualenvs/rl/lib/python3.5/site-packages/ipykernel_launcher.py:20: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.


In [8]:
#Buduję pierwszy model - w callbacku korzystam z metody
#PlotLossesKeras(), która umożliwi nam tworzenie interesujących nas
#wykresów danych metryk w czasie rzeczywistym
#korzystam również z early stopping zdefiniowanego wyżej, aby nie dopuścić
#do przetrenowania modelu
model2 = Sequential()
model2.add(Dense(20, activation='relu', input_shape=(10,)))


model2.add(Dropout(0.1,))
model2.add(Dense(75, activation='relu'))


model2.add(Dense(1, activation='sigmoid'))


model2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[keras_metrics.precision(),'accuracy'])
                   
model2.fit(X_train_std2, y_train2, epochs=20, batch_size=2,
           verbose=1, validation_split=0.25, callbacks = [PlotLossesKeras(), es], class_weight=class_weight)

Train on 6000 samples, validate on 2000 samples
Epoch 1/20


InternalError: Blas GEMM launch failed : a.shape=(2, 10), b.shape=(10, 20), m=2, n=20, k=10
	 [[Node: dense_1/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_dense_1_input_0_0/_75, dense_1/kernel/read)]]
	 [[Node: loss/mul/_143 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_728_loss/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'dense_1/MatMul', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/seba/.virtualenvs/rl/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/seba/.virtualenvs

In [ ]:
#Sprawdźmy predykcję i wygenerujmy raport klasyfikacyjny
y_pred2 = model2.predict_classes(X_test_std2)

report = classification_report(y_test2, y_pred2, output_dict=False)
print(report)

### autor: Paweł Bogdan